In [2]:
!git clone https://github.com/cg123/mergekit.git
%cd mergekit
%pip install -e .

fatal: destination path 'mergekit' already exists and is not an empty directory.
/content/mergekit
Obtaining file:///content/mergekit
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for mergekit (pyproject.toml) ... done
  Created wheel for mergekit: filename=mergekit-0.0.3.2-0.editable-py3-none-any.whl size=9027 sha256=3945b6f4b751a7e68352cedf8cbee98487af3c07ec77c5a1ca629ebb4b99190a
  Stored in directory: /tmp/pip-ephem-wheel-cache-18amcx60/wheels/ed/19/4a/2c5e8cd746584b45b047bcbc1cbf1cc31ccf037775edec8120
Successfully built mergekit
  Attempting uninstall: mergekit
    Found existing installation: mergekit 0.0.3.2
    Uninstalling mergekit-0.0.3.2:
      Successfully uninstalled mergekit-0.0.3.2


###Merging with SLERP

In [13]:
import yaml

MODEL_NAME = "Open-Phi-7B"
yaml_config = """

slices:
  - sources:
      - model: openchat/openchat-3.5-0106
        layer_range: [0, 32]
      - model: mrm8488/phi-2-coder
        layer_range: [0, 32]

merge_method: slerp
base_model: mistralai/Mistral-7B-v0.1

parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5 # fallback for rest of tensors
tokenizer_source: union

dtype: bfloat16
"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)

In [15]:
# Merge models
!mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle --trust-remote-code

configuration_phi.py: 100% 9.26k/9.26k [00:00<00:00, 30.5MB/s]
A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
You are using a model of type phi-msft to instantiate a model of type phi. This is not supported for all configurations of models and can yield errors.
tokenizer_config.json: 100% 1.62k/1.62k [00:00<00:00, 8.11MB/s]
tokenizer.model: 100% 493k/493k [00:00<00:00, 4.81MB/s]
tokenizer.json: 100% 1.80M/1.80M [00:00<00:00, 14.1MB/s]
added_tokens.json: 100% 53.0/53.0 [00:00<00:00, 297kB/s]
special_tokens_map.json: 100% 491/491 [00:00<00:00, 2.92MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
tokenizer_config.json: 100% 7.41k/7.41k [00:00<00:00, 22.2MB/s]
vocab.json: 100% 999k/999k [00:00<00:

###Create Model Card

###Merging with DARE TIES

In [9]:
import yaml

MODEL_NAME = "LelaWizard-7B"
yaml_config = """

models:
  - model: mistralai/Mistral-7B-v0.1
    # no parameters necessary for base model
  - model: SanjiWatsuki/Lelantos-DPO-7B
    parameters:
      density: 0.5
      weight: 0.5
  - model: deepseek-ai/deepseek-coder-6.7b-instruct
    parameters:
      density: 0.5
      weight: 0.5
merge_method: ties
base_model: mistralai/Mistral-7B-v0.1
parameters:
  normalize: true
dtype: float16
"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)

In [10]:
# Merge models
!mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle

config.json: 100% 571/571 [00:00<00:00, 1.73MB/s]
Fetching 11 files: 100% 11/11 [00:00<00:00, 13913.55it/s]
Fetching 8 files: 100% 8/8 [00:00<00:00, 8758.66it/s]
Fetching 10 files:   0% 0/10 [00:00<?, ?it/s]
model.safetensors.index.json: 100% 25.1k/25.1k [00:00<00:00, 7.60MB/s]

tokenizer.json:   0% 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json: 100% 72.0/72.0 [00:00<00:00, 284kB/s]


pytorch_model.bin.index.json: 100% 23.9k/23.9k [00:00<00:00, 51.8MB/s]


generation_config.json: 100% 116/116 [00:00<00:00, 621kB/s]
Fetching 10 files:  20% 2/10 [00:00<00:03,  2.05it/s]

tokenizer_config.json: 100% 967/967 [00:00<00:00, 3.25MB/s]

tokenizer.json: 100% 1.80M/1.80M [00:00<00:00, 10.7MB/s]

model-00001-of-00002.safetensors:   0% 0.00/9.94G [00:00<?, ?B/s]

tokenizer.model:   0% 0.00/493k [00:00<?, ?B/s]


model-00002-of-00002.safetensors:   0% 0.00/4.54G [00:00<?, ?B/s]

tokenizer.model: 100% 493k/493k [00:00<00:00, 4.80MB/s]

model-00001-of-00002.safetensors:   0% 10.5M/9.94G [00:00<

In [11]:
!pip install -qU huggingface_hub

from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template

username = "luqmanxyz"

template_text = """
---
license: apache-2.0
tags:
- merge
- mergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a merge of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

```yaml
{{- yaml_config -}}
```
"""

# Create a Jinja template object
jinja_template = Template(template_text.strip())

# Get list of models from config
data = yaml.safe_load(yaml_config)
if "models" in data:
    models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
elif "parameters" in data:
    models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
elif "slices" in data:
    models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
else:
    raise Exception("No models or slices found in yaml config")

# Fill the template
content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
)

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')

In [12]:
from google.colab import userdata
from huggingface_hub import HfApi

username = "luqmanxyz"

# Defined in the secrets tab in Google Colab
api = HfApi(token=userdata.get("hf"))

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)

CommitInfo(commit_url='https://huggingface.co/luqmanxyz/LelaWizard-7B/commit/c5057e9b45fd0a81e13895db1f3b0b18b2ec93ea', commit_message='Upload folder using huggingface_hub', commit_description='', oid='c5057e9b45fd0a81e13895db1f3b0b18b2ec93ea', pr_url=None, pr_revision=None, pr_num=None)